Setup and Install Dependencies: Ensure you have TensorFlow and other necessary libraries installed.

In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import Libraries:

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/Users/raysonyong/Documents/GitHub/csc3107-group26-p3-ml-brain-mri-classification/dataset_26'

Load Pre-trained DenseNet Model: Use DenseNet121, 169, or 201 as per your requirement. Here, we'll use DenseNet121.

In [3]:
# Load the DenseNet121 model pre-trained on ImageNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False


Building Model

In [4]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes: glioma, meningioma, no tumor, pituitary
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Prepare Your Data: Use ImageDataGenerator for data augmentation.



In [5]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of the data for validation
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


Train the model

In [6]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator
)


Epoch 1/25


/Users/raysonyong/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.3374 - loss: 1.8838 - val_accuracy: 0.5521 - val_loss: 1.0498
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.6224 - loss: 1.0095 - val_accuracy: 0.6354 - val_loss: 0.8471
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.7051 - loss: 0.7274 - val_accuracy: 0.6562 - val_loss: 0.8056
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.7719 - loss: 0.6362 - val_accuracy: 0.6979 - val_loss: 0.7521
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.7337 - loss: 0.6073 - val_accuracy: 0.6458 - val_loss: 0.7244
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8058 - loss: 0.5292 - val_accuracy: 0.7188 - val_loss: 0.6614
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.7654 - loss: 0.5917 - val_accuracy: 0.7604 - val_loss: 0.6405
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8118 - loss: 0.4944 - val_accuracy: 0.8021 - val_loss: 0.5264
Epo

Evaluate the model

In [7]:

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.8281 - loss: 0.4766
Test accuracy: 0.84
